In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from  sklearn import metrics
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from collections import Counter

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm , tree
from sklearn.ensemble import RandomForestClassifier

In [4]:
df=pd.read_csv('Original_Dataset.csv')
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
col=[]
for i in df.columns:
    if i != 'Disease':
        col.append(i)

In [6]:
symptoms= df.iloc[:,1:].values.flatten()
symptoms=list(set(symptoms))

In [7]:
for sym in symptoms:
    df[sym]=df.iloc[:,1:].apply(lambda row: int(sym in row.values),axis=1)
    
df1=df.drop(columns=col)

In [8]:
df1=df1.loc[:,df1.columns.notna()]

In [9]:
df1.columns= df1.columns.str.strip()

In [10]:
df1.columns

Index(['Disease', 'swollen_legs', 'chest_pain', 'silver_like_dusting',
       'shivering', 'coma', 'chills', 'fluid_overload', 'vomiting',
       'diarrhoea',
       ...
       'altered_sensorium', 'fast_heart_rate', 'fatigue', 'patches_in_throat',
       'polyuria', 'belly_pain', 'internal_itching', 'movement_stiffness',
       'sweating', 'nausea'],
      dtype='object', length=132)

In [11]:
var=['Disease']
le=LabelEncoder()
for j in var:
    df1[j]= le.fit_transform(df1[j])

In [12]:
x=df1.drop(['Disease'],axis=1)
y=df1['Disease']

In [13]:
doctor = pd.read_csv('Doctor_Versus_Disease.csv',encoding='latin1',names=['Disease','Specialist'])
doctor.head()

,Disease,Specialist
0,Drug Reaction,Allergist
1,Allergy,Allergist
2,Hypertension,Cardiologist
3,Heart attack,Cardiologist
4,Psoriasis,Dermatologist


In [14]:
doctor.tail()

,Disease,Specialist
36,Bronchial Asthma,Pulmonologist
37,Pneumonia,Pulmonologist
38,Osteoarthristis,Rheumatologists
39,Arthritis,Rheumatologists
40,Tuberculosis,Pulmonologist


In [15]:
def class_algo(model,independent,dependent):
    model.fit(independent,dependent)
    pred = model.predict(independent)
    accuracy = metrics.accuracy_score(pred,dependent)
    print(model_name,'Accuracy : %s' % '{0:.3%}'.format(accuracy))

lr=LogisticRegression()
lr.fit(x,y)

In [16]:
algorithms = {'Logistic Regression': 
              {"model": LogisticRegression()},
              
              'Decision Tree': 
              {"model": tree.DecisionTreeClassifier()},
              
              'Random Forest': 
              {"model": RandomForestClassifier()},
              
              'SVM':
              {"model": svm.SVC(probability=True)},
              
              'NaiveBayes' :
              {"model": GaussianNB()},
              
              'K-Nearest Neighbors' :
              {"model": KNeighborsClassifier()},
             }

In [17]:
for model_name, values in algorithms.items():
    class_algo(values["model"],x,y)

Logistic Regression Accuracy : 100.000%
Decision Tree Accuracy : 100.000%
Random Forest Accuracy : 100.000%
SVM Accuracy : 100.000%
NaiveBayes Accuracy : 100.000%
K-Nearest Neighbors Accuracy : 100.000%


In [18]:
disease=pd.read_csv('Disease_Description.csv')
disease.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [21]:
test_col = []
for col in df1.columns:
    if col != 'Disease':
        test_col.append(col)


test_data = {}
symptoms = []
predicted = []
def test_input():
    symptoms.clear()
    predicted.clear()
    num_inputs = int(input("Enter the number of symptoms you have: "))
    for i in range(num_inputs):
        user_input = input("Enter Symptoms #{}: ".format(i+1))
        symptoms.append(user_input)
    print("Symptoms you have:", symptoms)
    for column in test_col:
        test_data[column] = 1 if column in symptoms else 0
        test_df = pd.DataFrame(test_data, index=[0])
    for model_name, values in algorithms.items():
        predict_disease = values["model"].predict(test_df)
        predict_disease = le.inverse_transform(predict_disease)
        predicted.extend(predict_disease)
    disease_counts = Counter(predicted)
    percentage_per_disease = {disease: (count / 6) * 100 for disease, count in disease_counts.items()}
    result_df = pd.DataFrame({"Disease": list(percentage_per_disease.keys())})
    result_df = result_df.merge(doctor, on='Disease', how='left')
    result_df = result_df.merge(disease, on='Disease', how='left')
    return result_df

In [31]:
test_input()

Symptoms you have: ['itching', 'sneeze']


,Disease,Specialist,Description
0,Fungal infection,Dermatologist,"In humans, fungal infections occur when an inv..."
1,AIDS,Osteopathic,Acquired immunodeficiency syndrome (AIDS) is a...
